In [15]:
import warnings
from gensim import matutils
warnings.filterwarnings('ignore')
from gensim.models import KeyedVectors
from urllib import request
import string
from nltk.corpus import brown
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.stem.porter import *
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
from nltk.corpus import wordnet
from nltk.corpus import wordnet as wn
from gensim.utils import simple_preprocess
from gensim import corpora
import re
from nltk.stem.porter import PorterStemmer  
from nltk import pos_tag
porter_stemmer = PorterStemmer() 
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('brown')
nltk.download('punkt')
nltk.download('stopwords')
stop_words = stopwords.words('english')
punc = '[,.!\')(-;?:''"``""]'
from nltk import sent_tokenize, word_tokenize
import pandas as pd
#word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

word_vectors = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home_lab_local/s1810410/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home_lab_local/s1810410/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     /home_lab_local/s1810410/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home_lab_local/s1810410/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home_lab_local/s1810410/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
import json, pickle
def load_dict(file):
    f = open(file, 'r')
    js =f.read()
    DIC = json.loads(js)
    f.close()
    return DIC

# 获取字典信息
f1 = open('corpus/GlossDict.txt', 'r') 
f2 = open('corpus/GlossDictExpand.txt', 'r')
dic = eval(f1.read())   # str into dict
dic2 = eval(f2.read())
glossDict = dic
ExpandglossDict = dic2
f1.close()
f2.close()

# 获取文章信息和文章ID
f3 = open('corpus/ID.txt', 'r')
f4 = open('corpus/INSTANCE.txt', 'r')
id_list = [] # 3944
article = [] # 3944
for line in f3.readlines():
    line = line.strip('\n')
    line = line.strip('"')
    id_list.append(line)

for art in f4.readlines():
    art = art.strip('\n')
    art = art.strip('[,],\\n,",')
    article.append(art)
f3.close()
f4.close()
f6 = open('corpus/GlossDictWITHpunc.txt', 'r') 
dic6 = eval(f6.read())   # str into dict
glossWithPunc = list(dic6)
f6.close()
EPGwithSTOP = open('corpus/EPGwithSTOP.txt', 'r') 
dicEPG = eval(EPGwithSTOP.read())  
EPGwith = list(dicEPG)                                 # wrod 保留stopwords； 额外信息没有stopwords
EPGwithSTOP.close()
# ----------corpus------------
with open(r"corpus/corp.txt", "r") as fo:
    out = fo.read()
    sentence_corpus = json.loads(out)

#with open(r'new-corpus/new-corpus-list-news-2010-1M.txt', 'rb') as fp:
#    sentence_corpus2 = pickle.load(fp)

#with open(r'new-corpus/new-corpus-wiki-2016-1M.txt', 'rb') as fp:
#    sentence_corpus3 = pickle.load(fp)
# dict
glossAND_stop = load_dict('./corpus/glossAND_stop_dict.txt') # 包含stopwords的词典信息
glossDict = dic                                              # 不包含stopwrods的词典信息
hypANDstop = load_dict('./corpus/hypANDstop_dict.txt')    # 加入额外信息的包含stopwords的词典信息
hypAND = load_dict('./corpus/hypAND_dict.txt')            # 加入额外信息的不包含stopwords的词典信息
EPGwith = list(dicEPG)        
print(len(sentence_corpus))
print('ok')

193228
ok


In [47]:
sentence_corpus[1000]

['there',
 'be',
 'no',
 'trace',
 'leave',
 'on',
 'it',
 'only',
 'the',
 'wood',
 'be',
 'still',
 'damp']

In [53]:
#sentence_corpus = sentence_corpus2 + sentence_corpus1 + sentence_corpus3

In [18]:
# Lemmatizer 词性还原
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_sentence(sentence):
    res = []
    lemmatizer = WordNetLemmatizer()
    for word, pos in pos_tag(word_tokenize(sentence)):
        wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
        res.append(lemmatizer.lemmatize(word, pos=wordnet_pos))
    return res

article = list(map(lambda x: ' '.join(lemmatize_sentence(x)), article))

# clean data with oov, punc, lower characters
def clean(word_list):
    new_sent = word_list
    new_sent = [word for word in new_sent if word in word_vectors.wv.vocab]
    new_sent = [word for word in new_sent if word not in punc]
    new_sent = [w.lower() for w in new_sent]
   #new_sent = [word for word in new_sent if word not in stop_words]
    return new_sent

# delete short sentences in corpus, lemmatize
def txt2wordlist(text):
    afterClean = []
    text = sent_tokenize(text)
    for i in text:
        i = lemmatize_sentence(i)
        i = clean(i)
        if len(i) > 5:
            afterClean.append(i)
    return afterClean

# find collocations for a target word in test data and windowsize words
def Find_collo(keyword,s,e,n):
    collocation = []
    KeyWord_art = []
    target = article[s:e]
    for art in target:
        new_sent = lemmatize_sentence(art)
        new_sent = [word for word in new_sent if word not in punc]
        new_sent = [word.lower() for word in new_sent]
        with_stop_sent = new_sent
        new_sent = clean(new_sent)

        key_index2 = [j for j,x in enumerate(with_stop_sent) if x.find(keyword)!=-1][0]
        key_index =[j for j,x in enumerate(new_sent) if x.find(keyword)!=-1][0]

        key_art = new_sent[(key_index-n):key_index] + new_sent[(key_index):(key_index+n+1)]
        key_collo = with_stop_sent[key_index2-5:key_index2+6] 

        #key_collo2 = new_sent[key_index-1] + ' ' new_sent[key_index]
        KeyWord_art.append(key_art)
        collocation.append(key_collo)


    return collocation, KeyWord_art

# extract surrounding texts for a word, n is window size
def surround_context(keyword,corpus,n):
    instance_list = []
    for sent in sentence_corpus:
       #key_index =[j for j,x in enumerate(sentence) if x.find(keyword)!=-1] #Fuzzy lookup
        key_index =[j for j,x in enumerate(sent) if x == keyword]
        if len(key_index) != 0:
            i = key_index[0]   
            new_list = sent[(i-n):i] + sent[(i):(i+n+1)]
            instance_list.append(new_list)         
    return instance_list

In [12]:
from collections import Counter
import re
noun_key_value = re.compile(r'[a-z]+[%][\d]+[:][\d]+[:][\d]+[:]+[a-z]*[:]*[\d\d]*')
def RE(S):
    tag = re.findall('\d\d+', S)[0]  # extraction number
    collo = re.sub(' [0-9]+','',S)  #  sub number
    return tag, collo

def RE2(S):
    tag = re.search(noun_key_value, S).group()
    collo = re.sub(noun_key_value,'',S)
    return tag, collo

In [11]:
# 为目标单词在corpus中找短语, 
# param: TEXT-> corpus sentecne list, Word-> target word, 
# a,b-> start end index in gloss dict, n-> n pairs for similarity compute
# T -> threshold score of similarity

def tag_collo(TEXT,Word,a,b,n,T):
    score_list = []
    answer_key_list = []
    Score = []
    L_colo = []
    i=0
    while i in range(len(TEXT)):
        sim_score = []
        for d in EPGwith[a:b]:
            #sim = word_vectors.n_similarity(d['gloss'],TEXT[i])   #修改过
            sim = SelectWordsSim(d['gloss'], TEXT[i], n) # ith gloss, ith text, nwords
            sim_score.append(sim)
        max_index = sim_score.index(max(sim_score))
        max_score_key = EPGwith[a:b][max_index]['key']  # change dict
        gloss_words = EPGwith[a:b][max_index]['gloss']  # change dict
        max_score = max(sim_score)
        dict={}.fromkeys(('collocation','tag','gloss')) 
        word_index = [j for j,x in enumerate(TEXT[i]) if x.find(Word)!=-1]
        if len(word_index)>0: 
            word_index = word_index[0]
            try:
                COLLO = TEXT[i][word_index-2:word_index+3]
                if max_score > T and len(COLLO)!=0:
                    score_list.append(max_score)
                    answer_key_list.append(str(max_score_key))
                    Score.append(max_score) 
                    dict['collocation'] = COLLO
                    dict['tag'] = max_score_key
                    dict['gloss'] = gloss_words #
                    L_colo.append(dict)
                i = i+1
            except Exception as e:
                i = i+1
        else:
            i = i+1
    return L_colo,Score

In [10]:
def Tag_collo_with_filter(word, corpus, n, gs, ge, T, PATTERN):
    SC = surround_context(word, corpus, n)
    SC_LEM = []
    for i in SC:
        SC_txt = ' '.join(i)
        SC_lem = lemmatize_sentence(SC_txt)
        SC_lem = clean(SC_lem)
        SC_LEM.append(SC_lem)
        
    collocations = tag_collo(SC_LEM, word, gs, ge, 8, T)[0]
    c_ = []
    for i in collocations:
        c_tag_d = {}
        a = i['collocation']
        a_tag = i['tag']
        if len(a) == 5:
            try:
                a.index(word)
            except:
                continue
            else:
                c1, c2, c3, c4, c5 = a[:3], a[1:3], a[1:4], a[2:4], a[2:]
                if len(c1)>=2: c_tag_d[' '.join(c5)] = a_tag
                if len(c2)>=2: c_tag_d[' '.join(c4)] = a_tag
                if len(c3)>=2: c_tag_d[' '.join(c3)] = a_tag
                if len(c4)>=2: c_tag_d[' '.join(c2)] = a_tag
                if len(c5)>=2: c_tag_d[' '.join(c1)] = a_tag
            c_.append(c_tag_d)
            
    list1 = []     # list1 is collocations without tag
    list2 = []     # list2 is collocations with tag
    for i in range(len(c_)):
        for k,v in c_[i].items():
            c = k+' '+v
            list1.append(k)
            list2.append(c)
    a=Counter(list1)
    b=Counter(list2)
    left = []    # (短语， 总共出现次数， 短语+意思， 该意思出现次数)
    for k,v in a.items():
        for k1,v1 in b.items():
            k_sub = PATTERN(k1)[1]
          #  if k == k_sub:       # verb
            if k+' '==k_sub:   # noun or adj
                left.append((k,v,k1,v1))
        
    def takeSecond(elem): # Sort by second element
        return elem[1]  
    left.sort(key=takeSecond, reverse=True)
    
    #LEFT = []
    #for i in left:
    #  if i[1]>=5 and i[3]/i[1] >= 0.6:
    #    LEFT.append(i[2])

    # new rate compute method = p(s|c)/p(s)
    key_list = []
    for ele in left:
        key = PATTERN(ele[2])[0]
        key_list.append(key)
    cont = Counter(key_list) # get all sense and make a dict
    
    LEFT=[]
    scores = []
    for ele in left:
        p_s_c = ele[3]/ele[1]
        #key = PATTERN(ele[2])[0]
        #sense_count = cont[key]
        #p_s = sense_count/sum(cont.values())
        rate =  p_s_c
        if rate > 0.4 and ele[3]>=6:    # (0.5 2 noun) (0.7 4 adj) (0.7, 3 verb)
            LEFT.append(ele[2]) 
            scores.append((ele[2], rate)) 
    DICT = {}
    for i,j in map(PATTERN, LEFT):    #v-> RE, n-> RE2
        DICT[j] = i

    verb_delted_list = [word+' what','n\'t '+word, word+' but', 'but '+word, word+' or','or '+word,'the '+word, word+' the', 'in '+word, 'a '+word, word+' a', word+' that', 'that '+word, word+' it', 'it '+word]
    noun_delted_list = ['an '+word+' ', word+' would ', 'be '+word+' ', word+' he', word+' this ', 'this '+word+' ', word+ ' be ', 'or '+word+' ', word+ ' i ', 'any '+word+' ', 'the '+word+' the ', 'it '+word+' ',word+ ' his ',word+' or ', word+' but ',word+' that ', 'the '+word+' ', word+' the ', 'in '+word+' ', word+' in ', 'at '+'word'+' ', 'for '+word+' ', word+' at ', word+' for ', 'it '+word+' ', word+' it ', 'a '+word+' ', word+' a', word+' or'] 
    
    for i in noun_delted_list:
        try:
            del DICT[i]
        except Exception as e:
            continue
    return DICT, scores, LEFT

In [18]:
word = 'target_word'
noun_delted_list = ['an '+word+' ', word+' would ', 'be '+word+' ', word+' he', word+' this ', 'this '+word+' ', word+ ' be ', 'or '+word+' ', word+ ' i ', 'any '+word+' ', 'the '+word+' the ', 'it '+word+' ',word+ ' his ',word+' or ', word+' but ',word+' that ', 'the '+word+' ', word+' the ', 'in '+word+' ', word+' in ', 'at '+'word'+' ', 'for '+word+' ', word+' at ', word+' for ', 'it '+word+' ', word+' it ', 'a '+word+' ', word+' a', word+' or']
for i in noun_delted_list:
    print(i)

an target_word 
target_word would 
be target_word 
target_word he
target_word this 
this target_word 
target_word be 
or target_word 
target_word i 
any target_word 
the target_word the 
it target_word 
target_word his 
target_word or 
target_word but 
target_word that 
the target_word 
target_word the 
in target_word 
target_word in 
at word 
for target_word 
target_word at 
target_word for 
it target_word 
target_word it 
a target_word 
target_word a
target_word or


In [9]:
# 计算相似度（topn pairs方法）
import heapq
def SelectWordsSim(Glo,Cont,n):
    total = []
    cont_word = []
    Glo = [word for word in Glo if word in word_vectors.vocab]
    Cont = [word for word in Cont if word in word_vectors.vocab]
    for word1 in Glo:
        for word2 in Cont:
            sim = word_vectors.similarity(word1,word2)
            cont_word.append(word2)
            total.append(sim)
    topn = heapq.nlargest(n,total)   # top n words
    new_words = []
    for i in topn:
        index = total.index(i)
        selected_word = cont_word[index]
        new_words.append(selected_word)
    new_score = word_vectors.n_similarity(Glo,new_words)
    return new_score

# if no collocation has been matched, tag sense-key by computing
def Tag_origi(article,gloss):
    scores = []
    for i in range(len(gloss)):
        glo = gloss[i]['gloss']
        sim = compute_sim_bert(glo, article)   # <- bert
        #sim = SelectWordsSim(glo, article, 8)  # <- 表现最好  HRWE
        #glo = [word for word in glo if word in word_vectors.vocab]
        #article = [word for word in article if word in word_vectors.vocab]
        #sim = word_vectors.n_similarity(glo,article)  # <- base
        scores.append(sim)
        index = scores.index(max(scores))
        tag = gloss[index]['key']

    return tag

In [1]:
#bert-embedding tes
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import tensorflow.keras.backend as K
from transformers import *
import tokenizers
import math 

gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
tf.config.experimental.set_visible_devices(devices=gpus[1:],device_type='GPU') 

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

def compute_sim_bert(glo, article):
    #取bert最后一层的输出, 做平均, 计算向量余弦相似度
    glo = ' '.join(glo)
    article = ' '.join(article)
    glo_id = tokenizer.encode(glo, return_tensors="tf")
    article_id = tokenizer.encode(article, return_tensors="tf")
    
    glo_out, article_out = model(glo_id), model(article_id)
    glo_vec, article_vec =  tf.squeeze(glo_out[0]), tf.squeeze(article_out[0])   # 减去数值为1的维度
    
    glo_avg, article_avg= tf.reduce_mean(glo_vec, 0), tf.reduce_mean(article_vec, 0)   # 计算向量均值
    score = cosine_similarity([glo_avg, article_avg])[0][1]
    return score
    
a = ['i', 'have', 'an', 'apple']
b  = ['i', 'like', 'you']
compute_sim_bert(a,b)
# bert--over-----


0.6948754

In [11]:
# all collo:

In [8]:
# 最终整合函数！！！！
def Compute_(Keyword, Collocations, Articles, GS, GE):
    target_texts=[]
    Colla=[]
    DataColla=[]
    DATA=()
    DataArt=[]
    Colla = Collocations 
    DataColla = Articles[0]  # collo in test data for target word
    DataArt = Articles[1] 

    AN = []   # answer key
    colloAN =[]  # answer tagged by collo
    record_id = []
    _COLLO = []
    for i,text in enumerate(DataColla):
    # inner loop should be interrupted when a collo is meet the same one 
    # and skip this time for outer loop; 这个循环检查过没有问题
        flag = False
        t1 = ' '.join(text[3:6])
        t2 = ' '.join(text[4:6])
        t3 = ' '.join(text[4:7])
        t4 = ' '.join(text[5:7])
        t5 = ' '.join(text[5:8])
        t_all = ' '.join(text[3:8])
        
        for k,v in Colla.items():
          #if k in t_all:
            if t1+' '==k or t2+' '==k or t3+' '==k or t4+' '==k or t5+' '==k:  # noun & adj
            #if t1==k or t2==k or t3==k or t4==k or t5==k:                      # verb
                AN.append(v)
                colloAN.append(v)
                record_id.append(i)
                _COLLO.append(k)
                flag = True
                break
        if flag == True:
            continue
        else:
            Tag = Tag_origi(DataArt[i], glossDict[GS:GE])#glossDict[GS:GE])  ## change dict!!  between verb and noun
            AN.append(Tag)
    return AN, colloAN, record_id, _COLLO  #修改过

In [2]:
def BATCH_compute(keyword, GS, GE, T, AS, AE):
    collocations, articles, answer = None, None, None 
    collocations = Tag_collo_with_filter(keyword, sentence_corpus, 8, GS,GE,T, RE2)[0]  # RE -> verb, RE2 -> noun/adj
    articles = Find_collo(keyword, AS, AE, 10)       # 10 -> noun,adj  9 -> verb
    answer = Compute_(keyword, collocations, articles, GS, GE)
    return answer[0], answer[1], answer[2], answer[3]

def BATCH_count(keyword, GS, GE, T, AS, AE):
    collocations, articles, answer = None, None, None 
    collocations = Tag_collo_with_filter(keyword, sentence_corpus, 8, GS,GE,T, RE)  # RE -> verb, RE2 -> noun/adj
    return collocations[0]#len(collocations[0]), collocations[3]

# number 
#verb: 87125 -> 229
# noun: 25410 -> 104
# adj: 7635 -> 165
# all : 120170  -> 498

In [157]:
# verb word colo num : 87125 ; filtered: 229
# noun word colo num : 25410; filtered: 136
# adj   7635  14

In [158]:
res = []
for i in LS_v:
    a = BATCH_count(i[2], i[3], i[4], 0.75, i[0], i[1])
    res.append(a)

In [24]:
import time
start = time.clock()
answer_file = []
answer_file_tag_with_collo = []
id_tag_with_collo = []
matched_collo = []

print('starting...')
for i in LS_adj:
    a = BATCH_compute(i[2], i[3], i[4], 1.0, i[0], i[1])  # 1.0 不要coll; (0.7 noun,adj)(0.75 verb)
    matched_collo.append(a[3])
    answer_file.append(a[0])
    answer_file_tag_with_collo.append(a[1])
    id_tag_with_collo.append(a[2])
    print("%s --> runned over!" % (str(i[2])))     
end = time.clock()
print('Exe time---:', (end-start)/60,'min')

starting...
different --> runned over!
hot --> runned over!
important --> runned over!
solid --> runned over!
Exe time---: 16.819820700000005 min


In [27]:
verb_file = 'corpus/verb-bert.txt'  # corpus/verb.txt
noun_file = 'corpus/noun-bert.txt'
adj_file = 'corpus/adj-bert.txt'
with open(adj_file,'w') as f:  # change
    for answer in answer_file:
        for key in answer:
            f.write(key)
            f.write('\n')

f_true = pd.read_csv('corpus/v_true.txt',sep=' ',names=['0','1','2','3','4'], header=None, engine='python')
f_noun_true = pd.read_csv('corpus/N_true.txt',sep=' ',names=['0','1','2','3','4'], header=None ,engine='python')
f_adj_true = pd.read_csv('corpus/all_adj_answer.txt',sep=' ',names=['0','1','2','3','4'],header=None,engine='python')

f_answer = pd.read_csv(verb_file, sep=' ', header=None, dtype={0:str})  #修改第0列数据类型为str
f_noun_answer = pd.read_csv(noun_file,sep =' ',header=None)
f_adj_answer = pd.read_csv(adj_file,sep = ' ',header=None)
sense_merge()

In [28]:
VERB_id = []
NOUN_id = []
ADJ_id = []
j = 0
for i in LS_adj:    # LS_n
    id = i[1] - i[0]
    a = [j, id+j]
    ADJ_id.append(a)# NOUN_id
    j = id+j

#print(' collo acc:',compute_acc(NOUN_id, f_noun_true, f_noun_answer),'\n',
#      'all acc:',newcompare(f_noun_true, f_noun_answer)/1570)    # 1698 / 1570

print(' collo acc:',compute_acc(ADJ_id, f_adj_true, f_adj_answer),'\n',
      'all acc:',newcompare(f_adj_true, f_adj_answer)/141)    # 1698 / 1570

#print(' collo acc:',compute_acc(VERB_id, f_true, f_answer),'\n',
#      'all acc:',newcompare(f_true, f_answer)/1698)    # 1698 / 1570

#verb_acc.append(newcompare(f_true, f_answer))

#NOUN
# ws hrwe(5, 0.505)(8, 0.521)(10, 0.534)
# ws base (5, 0.513, 8,0.512,  10,0.506)
# Noun collocation 0.631
# expandas dic info 0.505 ; 

#verb:
# hrwe window-size 5: 0.563  8: 0.572 9: 0.583  10: 0.572
# base  ws 5: 0.567   8: 0.548  9: 0.541   10: 0.533

# -----------exp gloss result(base)----------
# exp -> 0.517 (verb)
# exp -> 0.447 (adj)
#---------bert result----------
# verb  0.431,  
# noun   0.494  , glove 0.462, 
# adj 0.518  ,   glove 0.496
#----------adj window----
# 5 -> 0.567, 8 -> 0.539 , 10 -> 0.560  (base)
# 5 -> 0.525, 8 -> 0.504 , 10 -> 0.511

 collo acc: 0 
 all acc: 0.5602836879432624


In [ ]:
#           (37->0.919)  0.540  6
#           (46->0.869)   0.539   5
#          (67->0.806)  0.541   4
# news-100 (86->0.67)  0.541    3  
#         (115->0.652) 0.538    2

In [ ]:
# big corpus 
# noun- col(117->0.628, all-acc: 0.536)    121 -> 0.653
# verb- col(340 -> 0.541, all-acc: 0.585, 0.75)  ;2949

# small corpus
# verb- col (221-> 0.548, all-acc: 0.587, 0.75) ; 373
# noun- col (68 -> 0.735, all-acc:0.545 , 0.7) ; 68


In [171]:
# noun-final
# ws 10 
#(base 0.506, 
# base+word collo 0.516   (T:0.7 word collo 0.5, 2)
# base+ depend collo 0.510 (depend collo 5 0.7)
# hrwe 0.534    
# hrwe + word collo 0.545
# hrwe + depd collo  0.540
# hrwe + word collo + hrwe collo  )
# collo setting: rate: 0.5, appera: 2 T: 0.7

#  collo only (word collo 0.63->279)


# verb - finale
# ws 9 (base, base+collo , hrwe , hrwe+collo ,)
# hrwe + wordcollo 0.587
# collo setting: rate 0.7, appear 3, T 0.75
# collo only 0.573 -> 618 ()
# ensemble 0.594

# adj-final
# ws 10, word collo: 0.7, 4  
#(base 0.560, 
# base+word collo 0.553 (T: 0.7   collo(num, acc)-> 6, 0.33), 
# base + depd collo 0.546( )
# hrwe 0.511 
# hrwe+ word collo 0.511 (T: 0.7 collo(num,acc)-> 6, 0.33)
# hrwe + depend collo 0.511 ( depend collo setting:5 0.7, depd-collo(num,acc)-> )
# hrwe + depd collo + word collo 0.511 
# collo 0.625  -> 16


#all (base 0.525, HRWE 0.557, hrwe+collo 0.564)
   

In [267]:
(0.553*1698 + 0.516* 1570 + 0.553*141) / (1698+1570+141)

0.5359598122616603

In [55]:
def AccSort(ID, TRUE, ANSWER, POS):
    #计算单个单词的精确度，观察固定搭配的熵是否有影响
    
    ret = []
    _,Colcorrect,Collen = comp_single(ID, TRUE, ANSWER)
    for k,i in enumerate(ID):
        word = POS[k][2]
        l,r = i[0], i[1]
        length = r-l
        res = newcompare(TRUE[l:r], ANSWER[l:r])/length         #1570  # accuracy for all noun instances
        collo_acc = Colcorrect[k] / (Collen[k]+0.001)
        rate = Collen[k] / length
        #ret.append([word, round(res,2), round(collo_acc,2), round(rate, 2)])
        ret.append([word, round(res, 2)])
    return ret

T = AccSort(VERB_id, f_true, f_answer, LS_v)

In [ ]:
# 计算collo only 方法需要用
all_id = []
for i,e in enumerate(VERB_id):
    if len(id_tag_with_collo[i]) != 0:
        for n in id_tag_with_collo[i]:
            new_n = n + e[0]
            all_id.append(new_n)
print(all_id, len(all_id))

In [8]:
LS_v = [(0,114,'activate',0,5),(114,246,'add',5,11),(246,379,'appear',11,14),(623,754,'ask',25,31),
     
        (1067,1146,'begin',51,55),(1146,1213,'climb',55,60),(1690,1777,'eat',89,96),
      
        (1777,1842,'encounter',96,100),
      
        (1975,2007,'hear',107,114),(2268,2304,'lose',162,171),(2304,2344,'mean',171,178),(2344,2374,'miss',178,186)
     
        ,(2919,2971,'play',221,233),(2971,3065,'produce',233,239),(3065,3134,'provide',239,245),
     
        (3134,3161,'receive',245,254),(3161,3231,'remain',254,257),(3231,3261,'rule',257,262),(3377,3432,'smell',273,280),
     
        (3589,3653,'suspend',305,312),(3653,3726,'talk',312,321),(3726,3783,'treat',321,330),(3783,3797,'use',330,335),
     
        (3797,3831,'wash',335,347),(3831,3882,'watch',347,354),(3882,3921,'win',354,361),(3921,3944,'write',361,369)]
      

LS_n = [(379,490,'argument',14,19),(490,623,'arm',19,25),
         
         (935,1067,'bank',41,51),(1275,1403,'degree',64,71),(1403,1517,'difference',71,76),(1567,1590,'difficulty',81,85),
         
         (1590,1690,'disc',85,89),(2050,2124,'image',136,143),(2143,2236,'interest',148,155),(2236,2268,'judgment',155,162),
         (2515,2632,'paper',201,208),(2632,2748,'party',208,213),(2748,2835,'performance',213,218),
         
         (2835,2919,'plan',218,221),(3261,3359,'shelter',262,266),(3461,3557,'sort',294,298),(3557,3589,'source',298,305)]
 

LS_adj = [(1517,1567,'different',76,81),(2007,2050,'hot',114,136),
          (2124,2143,'important',143,148),(3432,3461,'solid',280,294)]

In [11]:
len(LS_adj)

4

In [ ]:
#verb: 27; noun: 17;  adj: 4;

In [5]:
for i in LS_adj:
    print(i[2]+', ')

different, 
hot, 
important, 
solid, 


In [5]:
def compute_acc(index, tru, ans):   
    # 计算每个单词的collo的acc
    LEN_collo = []
    correct_number = []
    for i in range(len(id_tag_with_collo)):
        INDEX = id_tag_with_collo[i]
        A = index[i] ##
        c = colloComapre(tru, ans, A[0], A[1], INDEX)  ##
        LEN_collo.append(len(INDEX))
        correct_number.append(c)
    if sum(LEN_collo) != 0:
        return sum(LEN_collo), sum(correct_number)/sum(LEN_collo)  # p for tagged by collo insatnces
    else:
        return 0

def newcompare(TRUE,ANSWER):
    correct = []
    for i in range(len(ANSWER)):
        if ANSWER.iloc[i][0] in TRUE.iloc[i][2:5].values:
            correct.append(1)
    return len(correct)

def colloComapre(TRUE, ANSWER, AS, AE, INDEX):
    newTrue = None
    newAnswer = None
    number = None

    newTrue = TRUE[AS:AE]
    newAnswer = ANSWER[AS:AE]
    number = []
    if len(INDEX) == 0:
        return 0
    else:    
        for index in INDEX:
            answer = newAnswer.iloc[index][0]
            Tr = newTrue.iloc[index][2:5].values
            if answer in Tr:
                number.append(1)
    return len(number)


def comp_single(ID, f_tr, f_an):
    LEN_collo = []
    correct_number = []
    for i in range(len(id_tag_with_collo)):
        INDEX = id_tag_with_collo[i]
        A = ID[i] ##
        c = colloComapre(f_tr, f_an, A[0], A[1], INDEX)  ##
        LEN_collo.append(len(INDEX))
        correct_number.append(c)
    acc = []
    for idx, i in enumerate(LEN_collo):
        if i != 0:
            ac = correct_number[idx]/i
            acc.append(ac)
        else:
            acc.append(0)
    return acc, correct_number, LEN_collo # CHANGE

In [6]:
#Noune  合并后平均意思3.58， 17个单词， 1570个例子
def sense_merge():
    f_noun_answer.replace('argument%1:10:00::','a',inplace=True)
    f_noun_answer.replace('argument%1:10:00::','a',inplace=True)
    f_noun_answer.replace('bank%1:06:00::','b',inplace=True)
    f_noun_answer.replace('bank%1:06:01::','b',inplace=True)
    f_noun_answer.replace('bank%1:14:00::','b',inplace=True)
    f_noun_answer.replace('bank%1:14:01::','b',inplace=True)

    f_noun_answer.replace('bank%1:17:00::','c',inplace=True)
    f_noun_answer.replace('bank%1:17:01::','c',inplace=True)
    f_noun_answer.replace('bank%1:17:02::','c',inplace=True)

    f_noun_answer.replace('bank%1:21:00::','d',inplace=True)
    f_noun_answer.replace('bank%1:21:01::','d',inplace=True)

    f_noun_answer.replace('degree%1:07:00::','e',inplace=True)
    f_noun_answer.replace('degree%1:07:01::','e',inplace=True)
    f_noun_answer.replace('degree%1:26:01::','e',inplace=True)

    f_noun_answer.replace('image%1:06:00::','f',inplace=True)
    f_noun_answer.replace('image%1:06:01::','f',inplace=True)

    f_noun_answer.replace('image%1:07:00::','g',inplace=True)
    f_noun_answer.replace('image%1:09:00::','g',inplace=True)
    f_noun_answer.replace('image%1:09:02::','g',inplace=True)

    f_noun_answer.replace('interest%1:04:01::','h',inplace=True)
    f_noun_answer.replace('interest%1:07:01::','h',inplace=True)
    f_noun_answer.replace('interest%1:07:02::','h',inplace=True)
    f_noun_answer.replace('interest%1:09:00::','h',inplace=True)
    f_noun_answer.replace('interest%1:14:00::','h',inplace=True)
    f_noun_answer.replace('interest%1:21:00::','i',inplace=True)
    f_noun_answer.replace('interest%1:21:03::','i',inplace=True)

    #------------------
    f_noun_answer.replace('judgment%1:04:00::','j',inplace=True)
    f_noun_answer.replace('judgment%1:04:02::','j',inplace=True)
    f_noun_answer.replace('judgment%1:10:00::','j',inplace=True)
    f_noun_answer.replace('judgment%1:07:00::','k',inplace=True)
    f_noun_answer.replace('judgment%1:09:00::','k',inplace=True)
    f_noun_answer.replace('judgment%1:09:01::','k',inplace=True)
    f_noun_answer.replace('judgment%1:09:04::','k',inplace=True)
    #----------------

    f_noun_answer.replace('paper%1:10:01::','l',inplace=True)
    f_noun_answer.replace('paper%1:10:02::','l',inplace=True)
    f_noun_answer.replace('paper%1:06:00::','m',inplace=True)
    f_noun_answer.replace('paper%1:10:03::','m',inplace=True)
    f_noun_answer.replace('paper%1:14:00::','m',inplace=True)

    f_noun_answer.replace('paper%1:10:00::','n',inplace=True)
    f_noun_answer.replace('paper%1:27:00::','n',inplace=True)

    f_noun_answer.replace('party%1:11:00::','o',inplace=True)
    f_noun_answer.replace('party%1:14:02::','o',inplace=True)
    f_noun_answer.replace('party%1:18:00::','o',inplace=True)

    f_noun_answer.replace('performance%1:04:00::','p',inplace=True)
    f_noun_answer.replace('performance%1:04:01::','p',inplace=True)
    f_noun_answer.replace('performance%1:10:00::','p',inplace=True)

    f_noun_answer.replace('performance%1:04:03::','q',inplace=True)
    f_noun_answer.replace('performance%1:22:00::','q',inplace=True)

    f_noun_answer.replace('plan%1:09:00::','r',inplace=True)
    f_noun_answer.replace('plan%1:09:01::','r',inplace=True)

    f_noun_answer.replace('shelter%1:06:00::','s',inplace=True)
    f_noun_answer.replace('shelter%1:06:01::','s',inplace=True)

    f_noun_answer.replace('shelter%1:21:00::','t',inplace=True)
    f_noun_answer.replace('shelter%1:26:00::','t',inplace=True)

    f_noun_answer.replace('source%1:10:00::','u',inplace=True)
    f_noun_answer.replace('source%1:10:01::','u',inplace=True)
    f_noun_answer.replace('source%1:18:00::','u',inplace=True)


    #
    #answer
    #=-----------------
    f_noun_true.replace('argument%1:10:00::','a',inplace=True)
    f_noun_true.replace('argument%1:10:00::','a',inplace=True)

    f_noun_true.replace('bank%1:06:00::','b',inplace=True)
    f_noun_true.replace('bank%1:06:01::','b',inplace=True)
    f_noun_true.replace('bank%1:14:00::','b',inplace=True)
    f_noun_true.replace('bank%1:14:01::','b',inplace=True)

    f_noun_true.replace('bank%1:17:00::','c',inplace=True)
    f_noun_true.replace('bank%1:17:01::','c',inplace=True)
    f_noun_true.replace('bank%1:17:02::','c',inplace=True)

    f_noun_true.replace('bank%1:21:00::','d',inplace=True)
    f_noun_true.replace('bank%1:21:01::','d',inplace=True)

    f_noun_true.replace('degree%1:07:00::','e',inplace=True)
    f_noun_true.replace('degree%1:07:01::','e',inplace=True)
    f_noun_true.replace('degree%1:26:01::','e',inplace=True)

    f_noun_true.replace('image%1:06:00::','f',inplace=True)
    f_noun_true.replace('image%1:06:01::','f',inplace=True)

    f_noun_true.replace('image%1:07:00::','g',inplace=True)
    f_noun_true.replace('image%1:09:00::','g',inplace=True)
    f_noun_true.replace('image%1:09:02::','g',inplace=True)

    f_noun_true.replace('interest%1:04:01::','h',inplace=True)
    f_noun_true.replace('interest%1:07:01::','h',inplace=True)
    f_noun_true.replace('interest%1:07:02::','h',inplace=True)
    f_noun_true.replace('interest%1:09:00::','h',inplace=True)
    f_noun_true.replace('interest%1:14:00::','h',inplace=True)
    f_noun_true.replace('interest%1:21:00::','i',inplace=True)
    f_noun_true.replace('interest%1:21:03::','i',inplace=True)

    #------------------
    f_noun_true.replace('judgment%1:04:00::','j',inplace=True)
    f_noun_true.replace('judgment%1:04:02::','j',inplace=True)
    f_noun_true.replace('judgment%1:10:00::','j',inplace=True)
    f_noun_true.replace('judgment%1:07:00::','k',inplace=True)
    f_noun_true.replace('judgment%1:09:00::','k',inplace=True)
    f_noun_true.replace('judgment%1:09:01::','k',inplace=True)
    f_noun_true.replace('judgment%1:09:04::','k',inplace=True)
    #----------------

    f_noun_true.replace('paper%1:10:01::','l',inplace=True)
    f_noun_true.replace('paper%1:10:02::','l',inplace=True)
    f_noun_true.replace('paper%1:06:00::','m',inplace=True)
    f_noun_true.replace('paper%1:10:03::','m',inplace=True)
    f_noun_true.replace('paper%1:14:00::','m',inplace=True)

    f_noun_true.replace('paper%1:10:00::','n',inplace=True)
    f_noun_true.replace('paper%1:27:00::','n',inplace=True)

    f_noun_true.replace('party%1:11:00::','o',inplace=True)
    f_noun_true.replace('party%1:14:02::','o',inplace=True)
    f_noun_true.replace('party%1:18:00::','o',inplace=True)

    f_noun_true.replace('performance%1:04:00::','p',inplace=True)
    f_noun_true.replace('performance%1:04:01::','p',inplace=True)
    f_noun_true.replace('performance%1:10:00::','p',inplace=True)

    f_noun_true.replace('performance%1:04:03::','q',inplace=True)
    f_noun_true.replace('performance%1:22:00::','q',inplace=True)

    f_noun_true.replace('plan%1:09:00::','r',inplace=True)
    f_noun_true.replace('plan%1:09:01::','r',inplace=True)

    f_noun_true.replace('shelter%1:06:00::','s',inplace=True)
    f_noun_true.replace('shelter%1:06:01::','s',inplace=True)

    f_noun_true.replace('shelter%1:21:00::','t',inplace=True)
    f_noun_true.replace('shelter%1:26:00::','t',inplace=True)

    f_noun_true.replace('source%1:10:00::','u',inplace=True)
    f_noun_true.replace('source%1:10:01::','u',inplace=True)
    f_noun_true.replace('source%1:18:00::','u',inplace=True)

    #Verb  合并后意思 4.07， 27个单词， 1698 个例子

    f_answer.replace('42602','a',inplace=True)
    f_answer.replace('42605','a',inplace=True)
    f_answer.replace('238101','b',inplace=True)
    f_answer.replace('238105','b',inplace=True)
    f_answer.replace('238102','c',inplace=True)
    f_answer.replace('238104','c',inplace=True)
    f_answer.replace('238106','c',inplace=True)  # delete 'begin'
    f_answer.replace('770001','d',inplace=True)
    f_answer.replace('770002','d',inplace=True)   #delete 'decide'
    f_answer.replace('770005','d',inplace=True)
    f_answer.replace('1297001','e',inplace=True)
    f_answer.replace('1297006','e',inplace=True)
    f_answer.replace('1297002','f',inplace=True)
    f_answer.replace('1297007','f',inplace=True)
    f_answer.replace('1353101','g',inplace=True)  #delete expect,express
    f_answer.replace('1353103','g',inplace=True)
    f_answer.replace('1892101','h',inplace=True)
    f_answer.replace('1892103','h',inplace=True)
    f_answer.replace('1892105','h',inplace=True)

    f_answer.replace('1892102','i',inplace=True)
    f_answer.replace('1892104','i',inplace=True)
    f_answer.replace('1892106','i',inplace=True)
    f_answer.replace('1892107','i',inplace=True)

    f_answer.replace('2439901','j',inplace=True)
    f_answer.replace('2439902','j',inplace=True)
    f_answer.replace('2439904','j',inplace=True)
    f_answer.replace('2439905','j',inplace=True)

    f_answer.replace('2439903','k',inplace=True)
    f_answer.replace('2439908','k',inplace=True)
    f_answer.replace('2439906','k',inplace=True)

    f_answer.replace('2555501','l',inplace=True)
    f_answer.replace('2555502','l',inplace=True)
    f_answer.replace('2555507','l',inplace=True)

    f_answer.replace('2555504','m',inplace=True)
    f_answer.replace('2555505','m',inplace=True)
    f_answer.replace('2555506','m',inplace=True)

    f_answer.replace('2644301','n',inplace=True)
    f_answer.replace('2644307','n',inplace=True)
    f_answer.replace('2644302','n',inplace=True)
    f_answer.replace('2644303','n',inplace=True)   # delete note

    f_answer.replace('3165218','o',inplace=True)
    f_answer.replace('3165217','o',inplace=True)
    f_answer.replace('3165213','o',inplace=True)

    f_answer.replace('3165220','p',inplace=True)
    f_answer.replace('3165214','p',inplace=True)

    f_answer.replace('3288301','q',inplace=True)
    f_answer.replace('3288306','q',inplace=True)

    f_answer.replace('3288302','r',inplace=True)
    f_answer.replace('3288305','r',inplace=True)

    f_answer.replace('3313901','s',inplace=True)
    f_answer.replace('3313906','s',inplace=True)
    f_answer.replace('3313902','t',inplace=True)
    f_answer.replace('3313905','t',inplace=True)

    f_answer.replace('3434801','u',inplace=True)
    f_answer.replace('3434806','u',inplace=True)

    f_answer.replace('3434803','v',inplace=True)
    f_answer.replace('3434807','v',inplace=True)

    f_answer.replace('3477801','w',inplace=True)
    f_answer.replace('3477803','w',inplace=True)

    f_answer.replace('3597906','x',inplace=True)
    f_answer.replace('3597907','x',inplace=True)
    f_answer.replace('3597911','x',inplace=True)

    f_answer.replace('3893501','y',inplace=True)
    f_answer.replace('3893505','y',inplace=True)
    f_answer.replace('3893507','y',inplace=True)
    f_answer.replace('3893508','y',inplace=True)

    f_answer.replace('3893502','z',inplace=True)
    f_answer.replace('3893503','z',inplace=True)
    f_answer.replace('3893509','z',inplace=True)

    f_answer.replace('4155302','aa',inplace=True)
    f_answer.replace('4155303','aa',inplace=True)
    f_answer.replace('4155304','aa',inplace=True)
    f_answer.replace('4155307','aa',inplace=True)

    f_answer.replace('4198501','bb',inplace=True)
    f_answer.replace('4198502','bb',inplace=True)
    f_answer.replace('4198503','bb',inplace=True)
    f_answer.replace('4198506','bb',inplace=True)
    f_answer.replace('4198507','bb',inplace=True)  

    f_answer.replace('4380101','cc',inplace=True)
    f_answer.replace('4380102','cc',inplace=True)
    f_answer.replace('4380103','cc',inplace=True)

    f_answer.replace('4380104','dd',inplace=True)
    f_answer.replace('4380105','dd',inplace=True)

    #--------------------------------------------
    f_answer.replace('4380106','ee',inplace=True)
    f_answer.replace('4380109','ee',inplace=True)

    f_answer.replace('4530702','ff',inplace=True)
    f_answer.replace('4530704','ff',inplace=True)

    f_answer.replace('4636102','gg',inplace=True) 
    f_answer.replace('4636101','gg',inplace=True) 
    f_answer.replace('4636107','gg',inplace=True) 
    f_answer.replace('4636108','gg',inplace=True) 

    f_answer.replace('4636103','hh',inplace=True) 
    f_answer.replace('4636104','hh',inplace=True) 
    f_answer.replace('4636110','hh',inplace=True) 
    f_answer.replace('4636111','hh',inplace=True) 
    f_answer.replace('4636112','hh',inplace=True) 
    #---------------


    f_answer.replace('4640501','ii',inplace=True) 
    f_answer.replace('4640502','ii',inplace=True) 
    f_answer.replace('4640507','ii',inplace=True)

    f_answer.replace('4640503','jj',inplace=True) 
    f_answer.replace('4640508','jj',inplace=True) 
    f_answer.replace('4640509','jj',inplace=True) 
    #------------------
    f_answer.replace('4711401','kk',inplace=True) 
    f_answer.replace('4711403','kk',inplace=True) 
    f_answer.replace('4711405','kk',inplace=True)

    f_answer.replace('4711404','ll',inplace=True) 
    f_answer.replace('4711406','ll',inplace=True) 

    #---------------------------------
    f_answer.replace('4753401','mm',inplace=True) 
    f_answer.replace('4753404','mm',inplace=True) 
    f_answer.replace('4753406','mm',inplace=True)

    f_answer.replace('4753403','nn',inplace=True) 
    f_answer.replace('4753407','nn',inplace=True)

    f_answer.replace('4753405','oo',inplace=True) 
    f_answer.replace('4753408','oo',inplace=True)

    #
    # 答案
    #---------------------------------------------------------
    f_true.replace('42602','a',inplace=True)
    f_true.replace('42605','a',inplace=True)
    f_true.replace('238101','b',inplace=True)
    f_true.replace('238105','b',inplace=True)
    f_true.replace('238102','c',inplace=True)
    f_true.replace('238104','c',inplace=True)
    f_true.replace('238106','c',inplace=True)  # delete 'begin'
    f_true.replace('770001','d',inplace=True)
    f_true.replace('770002','d',inplace=True)   #delete 'decide'
    f_true.replace('770005','d',inplace=True)
    f_true.replace('1297001','e',inplace=True)
    f_true.replace('1297006','e',inplace=True)
    f_true.replace('1297002','f',inplace=True)
    f_true.replace('1297007','f',inplace=True)
    f_true.replace('1353101','g',inplace=True)  #delete expect,express
    f_true.replace('1353103','g',inplace=True)
    f_true.replace('1892101','h',inplace=True)
    f_true.replace('1892103','h',inplace=True)
    f_true.replace('1892105','h',inplace=True)
    f_true.replace('1892102','i',inplace=True)
    f_true.replace('1892104','i',inplace=True)
    f_true.replace('1892106','i',inplace=True)
    f_true.replace('1892107','i',inplace=True)
    f_true.replace('2439901','j',inplace=True)
    f_true.replace('2439902','j',inplace=True)
    f_true.replace('2439904','j',inplace=True)
    f_true.replace('2439905','j',inplace=True)
    f_true.replace('2439903','k',inplace=True)
    f_true.replace('2439908','k',inplace=True)
    f_true.replace('2439906','k',inplace=True)
    f_true.replace('2555501','l',inplace=True)
    f_true.replace('2555502','l',inplace=True)
    f_true.replace('2555507','l',inplace=True)
    f_true.replace('2555504','m',inplace=True)
    f_true.replace('2555505','m',inplace=True)
    f_true.replace('2555506','m',inplace=True)
    f_true.replace('2644301','n',inplace=True)
    f_true.replace('2644307','n',inplace=True)
    f_true.replace('2644302','n',inplace=True)
    f_true.replace('2644303','n',inplace=True)   # delete note
    f_true.replace('3165218','o',inplace=True)
    f_true.replace('3165217','o',inplace=True)
    f_true.replace('3165213','o',inplace=True)
    f_true.replace('3165220','p',inplace=True)
    f_true.replace('3165214','p',inplace=True)
    f_true.replace('3288301','q',inplace=True)
    f_true.replace('3288306','q',inplace=True)
    f_true.replace('3288302','r',inplace=True)
    f_true.replace('3288305','r',inplace=True)
    f_true.replace('3313901','s',inplace=True)
    f_true.replace('3313906','s',inplace=True)
    f_true.replace('3313902','t',inplace=True)
    f_true.replace('3313905','t',inplace=True)
    f_true.replace('3434801','u',inplace=True)
    f_true.replace('3434806','u',inplace=True)
    f_true.replace('3434803','v',inplace=True)
    f_true.replace('3434807','v',inplace=True)
    f_true.replace('3477801','w',inplace=True)
    f_true.replace('3477803','w',inplace=True)
    f_true.replace('3597906','x',inplace=True)
    f_true.replace('3597907','x',inplace=True)
    f_true.replace('3597911','x',inplace=True)
    f_true.replace('3893501','y',inplace=True)
    f_true.replace('3893505','y',inplace=True)
    f_true.replace('3893507','y',inplace=True)
    f_true.replace('3893508','y',inplace=True)
    f_true.replace('3893502','z',inplace=True)
    f_true.replace('3893503','z',inplace=True)
    f_true.replace('3893509','z',inplace=True)
    f_true.replace('4155302','aa',inplace=True)
    f_true.replace('4155303','aa',inplace=True)
    f_true.replace('4155304','aa',inplace=True)
    f_true.replace('4155307','aa',inplace=True)
    f_true.replace('4198501','bb',inplace=True)
    f_true.replace('4198502','bb',inplace=True)
    f_true.replace('4198503','bb',inplace=True)
    f_true.replace('4198506','bb',inplace=True)
    f_true.replace('4198507','bb',inplace=True)  
    f_true.replace('4380101','cc',inplace=True)
    f_true.replace('4380102','cc',inplace=True)
    f_true.replace('4380103','cc',inplace=True)
    f_true.replace('4380104','dd',inplace=True)
    f_true.replace('4380105','dd',inplace=True)

    #--------------------------------------------
    f_true.replace('4380106','ee',inplace=True)
    f_true.replace('4380109','ee',inplace=True)
    f_true.replace('4530702','ff',inplace=True)
    f_true.replace('4530704','ff',inplace=True)

    f_true.replace('4636102','gg',inplace=True) 
    f_true.replace('4636101','gg',inplace=True) 
    f_true.replace('4636107','gg',inplace=True) 
    f_true.replace('4636108','gg',inplace=True) 
    f_true.replace('4636103','hh',inplace=True) 
    f_true.replace('4636104','hh',inplace=True) 
    f_true.replace('4636110','hh',inplace=True) 
    f_true.replace('4636111','hh',inplace=True) 
    f_true.replace('4636112','hh',inplace=True) 
    #---------------

    f_true.replace('4640501','ii',inplace=True) 
    f_true.replace('4640502','ii',inplace=True) 
    f_true.replace('4640507','ii',inplace=True)
    f_true.replace('4640503','jj',inplace=True) 
    f_true.replace('4640508','jj',inplace=True) 
    f_true.replace('4640509','jj',inplace=True) 
    #------------------
    f_true.replace('4711401','kk',inplace=True) 
    f_true.replace('4711403','kk',inplace=True) 
    f_true.replace('4711405','kk',inplace=True)

    f_true.replace('4711404','ll',inplace=True) 
    f_true.replace('4711406','ll',inplace=True) 

    #---------------------------------
    f_true.replace('4753401','mm',inplace=True) 
    f_true.replace('4753404','mm',inplace=True) 
    f_true.replace('4753406','mm',inplace=True)
    f_true.replace('4753403','nn',inplace=True) 
    f_true.replace('4753407','nn',inplace=True)
    f_true.replace('4753405','oo',inplace=True) 
    f_true.replace('4753408','oo',inplace=True)



    #ADJ  合并后意思 2.75， 141个例子， 4个单词

    f_adj_answer.replace('hot%3:00:01::','a',inplace=True)
    f_adj_answer.replace('hot%3:00:02::','a',inplace=True)
    f_adj_answer.replace('hot%5:00:00:active:01','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:charged:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:fast:01','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:fresh:01','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:good:01','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:illegal:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:lucky:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:near:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:new:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:popular:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:pungent:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:radioactive:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:sexy:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:skilled:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:unpleasant:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:violent:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:wanted:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:warm:03','b',inplace=True)
    f_adj_answer.replace('hot%5:00:02:fast:01','b',inplace=True)
    f_adj_answer.replace('important%3:00:00::','c',inplace=True)
    f_adj_answer.replace('important%3:00:02::','c',inplace=True)
    f_adj_answer.replace('important%3:00:04::','c',inplace=True)
    f_adj_answer.replace('important%5:00:00:immodest:02','d',inplace=True)
    f_adj_answer.replace('important%5:00:00:influential:00','d',inplace=True)
    f_adj_answer.replace('solid%3:00:01::','e',inplace=True)
    f_adj_answer.replace('solid%3:00:02::','e',inplace=True)
    f_adj_answer.replace('solid%5:00:00:homogeneous:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:honorable:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:opaque:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:plain:02','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:sound:01','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:unbroken:02','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:undiversified:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:wholesome:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:hard:01','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:good:01','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:frozen:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:cubic:00','f',inplace=True)

    #
    #answer
    #-----------------

    f_adj_true.replace('hot%3:00:01::','a',inplace=True)
    f_adj_true.replace('hot%3:00:02::','a',inplace=True)
    f_adj_true.replace('hot%5:00:00:active:01','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:charged:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:fast:01','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:fresh:01','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:good:01','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:illegal:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:lucky:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:near:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:new:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:popular:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:pungent:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:radioactive:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:sexy:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:skilled:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:unpleasant:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:violent:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:wanted:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:warm:03','b',inplace=True)
    f_adj_true.replace('hot%5:00:02:fast:01','b',inplace=True)
    f_adj_true.replace('important%3:00:00::','c',inplace=True)
    f_adj_true.replace('important%3:00:02::','c',inplace=True)
    f_adj_true.replace('important%3:00:04::','c',inplace=True)
    f_adj_true.replace('important%5:00:00:immodest:02','d',inplace=True)
    f_adj_true.replace('important%5:00:00:influential:00','d',inplace=True)
    f_adj_true.replace('solid%3:00:01::','e',inplace=True)
    f_adj_true.replace('solid%3:00:02::','e',inplace=True)
    f_adj_true.replace('solid%5:00:00:homogeneous:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:honorable:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:opaque:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:plain:02','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:sound:01','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:unbroken:02','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:undiversified:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:wholesome:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:hard:01','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:good:01','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:frozen:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:cubic:00','f',inplace=True)
